In [ ]:
# import mantid algorithms, numpy and matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 8]
#%matplotlib notebook

from mantid.simpleapi import *

import numpy as np
from mantid import plots
from mantid.simpleapi import CreateWorkspace
import smooth_data
import scipp as sc
from scipp import Dim

## DAS-148 Demo
This notebook shows that a developed scipp function for smoothing data is eqvivalent to the original SmoothData found in Mantid. A random dataset is generated from a gaussian distribution with gaussian errors scaled by the squareroot of the signal. A second dataset with a few outliers is generated as well to see how the smoothing handle such cases. Data is smoothed using both solutions and the results are plotted.

In [ ]:
# Create a workspace that has a Gaussian peak, Mantid plot example
x = np.arange(30)
y0 = 20.+50.*np.exp(-(x-8.)**2./120.)
err = np.sqrt(y0)
y = 20.+50.*np.exp(-(x-8.)**2./120.)
y += err*np.random.normal(size=len(err))
err = np.sqrt(y)

# Add a dataset with outliers
y_outlier = y + np.around(np.random.sample(size=len(err))*0.54)*45.0
err_outlier = np.sqrt(y_outlier)

# Create Mantid workspaces for easy plotting
w = CreateWorkspace(DataX=x, DataY=y, DataE=err, NSpec=1, UnitX='tof')
w_outlier = CreateWorkspace(DataX=x, DataY=y_outlier, DataE=err_outlier, NSpec=1, UnitX='tof')

In [ ]:
def plot_comparison_mantid(w, x, y0, y, err, NPoints):
    fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
    ax.errorbar(w,'rs', label='Original data') # plot the workspace with errorbars, using red squares
    ax.plot(x, y0,'k-', label='Base distribution') # plot the initial distribution with black line

    smooth = SmoothData(w, NPoints)
    ax.errorbar(smooth,'bo', label='Smoothed data') # plot the workspace with errorbars, using blue circles
    ax.legend(['Original distribution', 'Generated data', 'Smoothed'])

### Reference Mantid plot, 3 and 5 point smoothing
Here we see a comparison between smoothing with 3 and 5 points using the reference Mantid routine.

In [ ]:
plot_comparison_mantid(w, x, y0, y, err, 3)
plot_comparison_mantid(w, x, y0, y, err, 5)

### Reference Mantid plot, 3 and 5 point smoothing of data with outliers

Here we see a comparison between smoothing with 3 and 5 points using the reference Mantid routine. A few outliers have been added to this dataset, which the smoothing has issues with.

In [ ]:
plot_comparison_mantid(w_outlier, x, y0, y_outlier, err_outlier, 3)
plot_comparison_mantid(w_outlier, x, y0, y_outlier, err_outlier, 5)

In [ ]:
def plot_comparison_scipp(w, x, y0, y, err, NPoints):
    fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
    ax.errorbar(w,'rs', label='Original data') # plot the workspace with errorbars, using red squares
    ax.plot(x, y0,'k-', label='Base distribution') # plot the initial distribution with black line
    
    input_y = sc.Variable(dims=[Dim.Tof], values=y, variances=err**2, unit=sc.units.us)
    output = smooth_data.smooth_data(input_y, dim=Dim.Tof, NPoints=NPoints)
    
    smooth = CreateWorkspace(DataX=x, DataY=output.values, DataE=np.sqrt(output.variances), NSpec=1, UnitX='Tof')

    ax.errorbar(smooth,'bo', label='Smoothed data') # plot the workspace with errorbars, using blue circles
    ax.legend(['Original distribution', 'Generated data', 'Smoothed'])

### Scipp plot, 3 and 5 point smoothing
Here we see a comparison between smoothing with 3 and 5 points with the developed scipp routine. Results are visually similar to the Mantid version as expected.

In [ ]:
plot_comparison_scipp(w, x, y0, y, err, 3)
plot_comparison_scipp(w, x, y0, y, err, 5)

### Scipp plot, 3 and 5 point smoothing of data with outliers

Here we see a comparison between smoothing with 3 and 5 points using the developed scipp routine. A few outliers have been added to this dataset, which the smoothing has issues with. The scipp routine seems to handle this in the same way as the reference.

In [ ]:
plot_comparison_scipp(w_outlier, x, y0, y_outlier, err_outlier, 3)
plot_comparison_scipp(w_outlier, x, y0, y_outlier, err_outlier, 5)

### Direct comparison
Here two identical plots are made using the two different approaches, Mantid and scipp.

In [ ]:
plot_comparison_mantid(w_outlier, x, y0, y_outlier, err_outlier, 5)
plot_comparison_scipp(w_outlier, x, y0, y_outlier, err_outlier, 5)

## Numerical test for identical results
Here the two different methods are used on the same data set, and instead of plotting the results the numpy routine allclose is used to check that the returned data is identical within reasonable tolerances.

In [ ]:
# Scipp smooth
input_y = sc.Variable(dims=[Dim.Tof], values=y, variances=err**2, unit=sc.units.us)
output = smooth_data.smooth_data(input_y, dim=Dim.Tof, NPoints=3)

# Mantid smooth reference
smooth = SmoothData(w, 3)

np.allclose(output.values, smooth.readY(0))

In [ ]:
np.allclose(np.sqrt(output.variances), smooth.readE(0))

In [ ]:
# Scipp smooth
output = smooth_data.smooth_data(input_y, dim=Dim.Tof, NPoints=5)

# Mantid smooth reference
smooth = SmoothData(w, 5)

np.allclose(output.values, smooth.readY(0))

In [ ]:
np.allclose(np.sqrt(output.variances), smooth.readE(0))